# Psycopg – wykonywanie SQL w Pythonie

Biblioteka służąca do wykonywania kodu SQL na bazie danych z poziomu Pythona. Jest przystosowana wyłącznie do pracy z systemem PostgreSQL. Inne systemy mają swoje odpowiedniki, np.:

- [MySQL](https://pypi.org/project/mysql-connector-python/)
- [Oracle](https://pypi.org/project/oracledb/)


Najnowsza wersja ma numer 3 ([PyPI](https://pypi.org/project/psycopg/)) chociaż cały czas popularna jest wersja 2 ([PyPI](https://pypi.org/project/psycopg2/)).

In [ ]:
import psycopg

In [ ]:
conn = psycopg.connect(dbname="postgres", user="postgres", password="postgres", host="localhost")
cursor = conn.cursor()

## Zapytania SQL

### CREATE

In [ ]:
create_query = """CREATE TABLE tasks(
    id SERIAL,
    description TEXT,
    priority SMALLINT,
    is_completed BOOLEAN
)
"""

In [ ]:
cursor.execute(create_query)
conn.commit()

In [ ]:
create_query = """CREATE TABLE users(
    id SERIAL,
    username VARCHAR(16),
    password VARCHAR(36),
    is_admin BOOLEAN
)
"""

In [ ]:
cursor.execute(create_query)
conn.commit()

### INSERT

In [ ]:
insert_query = "INSERT INTO tasks(description, priority, is_completed) VALUES('Opis zadania', 2, true)"

In [ ]:
cursor.execute(insert_query)
conn.commit()

#### Klauzula `RETURNING`

In [ ]:
insert_query = """INSERT INTO tasks (description, priority, is_completed)
                  VALUES ('Zrobić obiad', 3, false)  RETURNING *"""

In [ ]:
cursor.execute(insert_query)
record = cursor.fetchone()
record

In [ ]:
conn.commit()

### SELECT

In [ ]:
select_query = "SELECT * FROM tasks"

In [ ]:
cursor.execute(select_query)
results = cursor.fetchall()

In [ ]:
results

**Wiersze jako słowniki**

In [ ]:
cursor = conn.cursor(row_factory=psycopg.rows.dict_row)

In [ ]:
cursor.execute(select_query)
results = cursor.fetchall()
results

In [ ]:
cursor = conn.cursor()

### UPDATE

In [ ]:
update_query = """UPDATE tasks SET description='Nauczyć się SQLa'
                  WHERE id=1 RETURNING *"""

In [ ]:
cursor.execute(update_query)
record = cursor.fetchone()
record

In [ ]:
conn.commit()

### DELETE

In [ ]:
delete_query = "DELETE FROM tasks WHERE id=2;"
cursor.execute(delete_query)
conn.commit()

### DROP

In [ ]:
drop_query = "DROP TABLE users;"

cursor.execute(drop_query)
conn.commit()

## Template'y zapytań i zapobieganie SQL injection 

In [ ]:
id_number = '1'

query = f"UPDATE tasks SET priority = 3 WHERE id = {id_number}"
query

In [ ]:
cursor.execute(query)
conn.commit()

---

In [ ]:
# przygotowanie tabeli do usunięcia

create_query = "CREATE TABLE test();"
cursor.execute(create_query)
conn.commit()

SQL injection:

In [ ]:
id_number = '1;  DROP TABLE test;'


query = f"UPDATE tasks SET priority = 4 WHERE id = {id_number}"
query

In [ ]:
cursor.execute(query)
conn.commit()

---

In [ ]:
id_number = '1'

query = "UPDATE tasks SET priority = 5 WHERE id = %s"

cursor.execute(query, (id_number,))
conn.commit()

Gdybyśmy chcieli zobaczyć podgląd takiego zapytania, możemy zrobić to korzystając z następującego feature'a pythonowych stringów:

In [ ]:
query % (id_number,)

---

In [ ]:
# create_query = "CREATE TABLE test();"
# cursor.execute(create_query)
# conn.commit()

In [ ]:
id_number = '1; DROP TABLE test;'

query = "UPDATE tasks SET priority = 6 WHERE id = %s"

cursor.execute(query, (id_number,))

In [ ]:
conn.rollback()

> **ZADANIA**

In [ ]:
cursor.close()
conn.close()